In [1]:

!pip install sqlalchemy


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, Date, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, Date, Float, func


Base = declarative_base()



C:\Users\haulmark\AppData\Local\Temp\ipykernel_12692\1344665909.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [14]:
class Category(Base):
    __tablename__ = 'Categories'
    
    CategoryID = Column(Integer, primary_key=True, autoincrement=True, extend_existing=True)
    CategoryName = Column(String, extend_existing=True)



# Users Table
class User(Base):
    __tablename__ = 'Users'
    
    UserID = Column(Integer, primary_key=True, autoincrement=True)
    FirstName = Column(String)
    LastName = Column(String)
    Email = Column(String)
    JoinDate = Column(Date)

# Recipes Table
class Recipe(Base):
    __tablename__ = 'Recipes'
    
    RecipeID = Column(Integer, primary_key=True, autoincrement=True)
    Title = Column(String)
    Description = Column(String)
    CookingTime = Column(Integer)
    Servings = Column(Integer)
    NumberOfSteps = Column(Integer)
    SubmittedDate = Column(Date)
    NumberOfIngredients = Column(Integer)
    ImageURL = Column(String)

# Ingredients Table
class Ingredient(Base):
    __tablename__ = 'Ingredients'
    
    IngredientsID = Column(Integer, primary_key=True, autoincrement=True)
    Ingredients = Column(String, nullable=False)
    ServingSize = Column(Integer)
    Calories = Column(Integer)
    TotalFat = Column(Integer)
    SaturatedFat = Column(Integer)
    Cholesterol = Column(Integer)
    Sodium = Column(Integer)
    Carbohydrate = Column(Integer)
    Fiber = Column(Integer)
    Sugars = Column(Integer)
    Protein = Column(Integer)
    Fat = Column(Integer)
    Caffeine = Column(Integer)
    Source = Column(String)
    Date = Column(String)
    IsAllergen = Column(String)
    Food_allergen_ID = Column(Integer)
    Food = Column(String)
    Food_Class = Column(String)
    Food_type = Column(String)
    Food_Group = Column(String)
    Allergy_Type = Column(String)

# Recipe_Ingredients Fact Table
class RecipeIngredientsFact(Base):
    __tablename__ = 'Recipe_Ingredients_fact_table'
    
    CompID = Column(Integer, primary_key=True, autoincrement=True)
    RecipeID = Column(Integer, ForeignKey('recipes.RecipeID'))
    IngredientsID = Column(Integer, ForeignKey('ingredients.IngredientsID'))
    Ingredients = Column(String)

# Reviews Table
class Review(Base):
    __tablename__ = 'Reviews'
    
    ReviewID = Column(Integer, primary_key=True, autoincrement=True)
    Rating = Column(Integer)
    ReviewText = Column(String)
    ReviewDate = Column(Date)
    UserID = Column(Integer, ForeignKey('users.UserID'))
    RecipeID = Column(Integer, ForeignKey('recipes.RecipeID'))

# Recipe_Categories Table
class RecipeCategory(Base):
    __tablename__ = 'Recipe_Categories'
    
    CategoryID = Column(Integer, ForeignKey('categories.CategoryID'), primary_key=True)
    RecipeID = Column(Integer, ForeignKey('recipes.RecipeID'), primary_key=True)


C:\Users\haulmark\AppData\Local\Temp\ipykernel_12692\756321538.py:4: SAWarning: Can't validate argument 'extend_existing'; can't locate any SQLAlchemy dialect named 'extend'
  CategoryID = Column(Integer, primary_key=True, autoincrement=True, extend_existing=True)
C:\Users\haulmark\AppData\Local\Temp\ipykernel_12692\756321538.py:5: SAWarning: Can't validate argument 'extend_existing'; can't locate any SQLAlchemy dialect named 'extend'
  CategoryName = Column(String, extend_existing=True)


In [15]:
# Set up the database connection and session (no need to create tables if they already exist)
engine = create_engine('sqlite:///my_recipes.db')

# Create a session to interact with the existing database
Session = sessionmaker(bind=engine)
session = Session()

print("Connected to the existing database.")


Connected to the existing database.


In [31]:
#Query 1 function to select from a group of recipe categories
# List of available categories
available_categories = ["Snacks", "Soups", "Desserts", "Main Courses", "Breads", "Salads", "Appetizers", "Poultry",
                        "Sauces", "Seafood", "Beverages", "Pork", "Beef", "Lamb", "Vegetarian", "Pasta", "Vegan"]

def check_category_exists(category_name):
    # Check if the category exists in the database
    category = session.query(Category).filter(Category.CategoryName == category_name).first()
    return category is not None

def get_recipes_by_category(category_name):
    # Fetch recipes and ingredients for a specific category
    results = session.query(Recipe, Ingredient, RecipeIngredientsFact).join(
        RecipeIngredientsFact, Recipe.RecipeID == RecipeIngredientsFact.RecipeID).join(
        Ingredient, RecipeIngredientsFact.IngredientsID == Ingredient.IngredientsID).join(
        RecipeCategory, Recipe.RecipeID == RecipeCategory.RecipeID).join(
        Category, RecipeCategory.CategoryID == Category.CategoryID).filter(
        Category.CategoryName == category_name).limit(10).all()

    return results

# Ask the user for a category name and validate it
category_name = input("Enter a category name: ")

while not check_category_exists(category_name):
    print(f"Category '{category_name}' is not available.")
    print("Please choose from the following available categories:")
    print(", ".join(available_categories))
    
    # Prompt the user to enter a valid category name
    category_name = input("Enter a valid category name: ")

# Fetch recipes and ingredients for the valid category
recipes_by_category = get_recipes_by_category(category_name)

# Check if there are recipes for the given category
if recipes_by_category:
    # If recipes exist, print the results
    for recipe, ingredient, _ in recipes_by_category:
        print(f"Recipe: {recipe.Title}, Ingredient: {ingredient.Ingredients}")
else:
    # If no recipes are available, notify the user
    print(f"No recipes found for the category '{category_name}'.")


No recipes found for the category 'Lamb'.


In [28]:
# Query 2 function to get the average rating for each recipe (Aggregated Data)
def get_average_rating_per_recipe():
    return session.query(Recipe.RecipeID, Recipe.Title, func.avg(Review.Rating).label('average_rating')).join(
        Review, Recipe.RecipeID == Review.RecipeID).group_by(Recipe.RecipeID).all()


In [32]:
# Query 2 call for Find a Recipe ID that has at least one review
recipe_with_reviews = session.query(Recipe.RecipeID).select_from(Recipe).join(Review, Recipe.RecipeID == Review.RecipeID).first()

# If a recipe with reviews exists, query its reviews
if recipe_with_reviews:
    recipe_id = recipe_with_reviews.RecipeID
    reviews = get_reviews_for_recipe(recipe_id)
    
    if reviews:
        for review in reviews:
            print(f"Review: {review.ReviewText}, Rating: {review.Rating}")
    else:
        print(f"No reviews found for RecipeID {recipe_id}")
else:
    print("No recipes with reviews found.")



Review: This recipe is okay., Rating: 3


In [20]:
# Query 3 function to get the average rating for each recipe (Aggregated Data)
def get_average_rating_per_recipe():
    return session.query(Recipe.RecipeID, Recipe.Title, func.avg(Review.Rating).label('average_rating')).join(
        Review, Recipe.RecipeID == Review.RecipeID).group_by(Recipe.RecipeID).all()

# Query 3 call to get the average rating for each recipe
average_ratings = get_average_rating_per_recipe()
for recipe_id, title, avg_rating in average_ratings:
    print(f"Recipe: {title}, Average Rating: {avg_rating}")


Recipe: Squash And Kale Risotto, Average Rating: 3.0
Recipe: Winter Squash Apple And Walnut Soup, Average Rating: 4.0
Recipe: Wonderful Winter Squash Risotto, Average Rating: 5.0
Recipe: Steph S Chicken Tortilla Soup, Average Rating: 1.0
Recipe: Cumberland Rum Butter, Average Rating: 5.0
